In [11]:
import requests
import numpy as np
import pandas as pd
from datetime import datetime, timedelta
import yfinance as yf
import matplotlib.pyplot as plt
from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras.models import load_model
from tensorflow.keras.losses import MeanSquaredError  
import talib

In [12]:
def fetch_latest_data():
    # Define the ticker symbol for Bitcoin
    btc_ticker = "BTC-USD"

    # Calculate the date 210 days ago (200 days of data + 10 days recent data)
    end_date = datetime.today().strftime('%Y-%m-%d')  # Today’s date
    start_date = (datetime.today() - timedelta(days=210)).strftime('%Y-%m-%d')  # 210 days ago

    # Fetch the data using yfinance
    btc_data = yf.download(btc_ticker, start=start_date, end=end_date)

    # Reset the index to make 'Date' a column
    btc_data.reset_index(inplace=True)

    # Select and reorder columns for consistency with your request
    btc_data = btc_data[["Date", "Open", "High", "Low", "Close", "Volume"]]

    # Format the 'Date' column to match the required structure
    btc_data["Date"] = btc_data["Date"].dt.strftime("%Y-%m-%d")
    return btc_data

# دریافت داده‌ها
df = fetch_latest_data()
df.tail(10)

c:\Users\hp\AppData\Local\Programs\Python\Python312\Lib\site-packages\yfinance\utils.py:775: FutureWarning: The 'unit' keyword in TimedeltaIndex construction is deprecated and will be removed in a future version. Use pd.to_timedelta instead.
  df.index += _pd.TimedeltaIndex(dst_error_hours, 'h')
[*********************100%%**********************]  1 of 1 completed
C:\Users\hp\AppData\Local\Temp\ipykernel_11316\1792981782.py:19: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  btc_data["Date"] = btc_data["Date"].dt.strftime("%Y-%m-%d")


,Date,Open,High,Low,Close,Volume
200,2025-02-02,100661.539062,101430.664062,96216.078125,97688.976562,63091816853
201,2025-02-03,97681.101562,102514.171875,91242.890625,101405.421875,115400897748
202,2025-02-04,101398.718750,101745.617188,96208.109375,97871.820312,73002130211
203,2025-02-05,97878.007812,99113.210938,96174.828125,96615.445312,49125911241
204,2025-02-06,96610.640625,99168.609375,95707.351562,96593.296875,45302471947
205,2025-02-07,96581.320312,100154.140625,95653.882812,96529.085938,55741290456
206,2025-02-08,96533.257812,96877.804688,95702.492188,96482.453125,22447526395
207,2025-02-09,96481.312500,97325.281250,94745.257812,96500.093750,27732901800
208,2025-02-10,96499.460938,98333.218750,95320.843750,97437.554688,40078962391
209,2025-02-11,97438.132812,98492.898438,94875.039062,95747.429688,37488783272


In [13]:
# -------------------------------
# 1. محاسبه میانگین‌های متحرک
# -------------------------------
df['MA_20'] = df['Close'].rolling(window=20).mean()
df['MA_50'] = df['Close'].rolling(window=50).mean()
df['MA_200'] = df['Close'].rolling(window=200).mean()

# -------------------------------
# 2. محاسبه شاخص قدرت نسبی (RSI)
# -------------------------------
period = 14  # دوره محاسبه RSI
delta = df['Close'].diff()

# محاسبه سود و زیان
gain = delta.clip(lower=0)
loss = -delta.clip(upper=0)

# محاسبه میانگین سود و زیان
avg_gain = gain.rolling(window=period, min_periods=period).mean()
avg_loss = loss.rolling(window=period, min_periods=period).mean()

# محاسبه RSI
rs = avg_gain / avg_loss
df['RSI'] = 100 - (100 / (1 + rs))

# رفع مقدار NaN با استفاده از انتساب مستقیم (برای رفع FutureWarning)
df['RSI'] = df['RSI'].fillna(0)

# -------------------------------
# 3. محاسبه MACD، سیگنال و هیستوگرام
# -------------------------------
# محاسبه EMA با دوره‌های 12 و 26
ema12 = df['Close'].ewm(span=12, adjust=False).mean()
ema26 = df['Close'].ewm(span=26, adjust=False).mean()

# MACD
df['MACD'] = ema12 - ema26

# خط سیگنال: EMA9 از MACD
df['Signal'] = df['MACD'].ewm(span=9, adjust=False).mean()

# هیستوگرام MACD: تفاوت MACD و خط سیگنال
df['Histogram'] = df['MACD'] - df['Signal']

# نمایش نمونه‌ای از دیتاست با ستون‌های اضافه شده
df.tail(10)  # نمایش 10 روز آخر



,Date,Open,High,Low,Close,Volume,MA_20,MA_50,MA_200,RSI,MACD,Signal,Histogram
200,2025-02-02,100661.539062,101430.664062,96216.078125,97688.976562,63091816853,102367.827734,99191.974844,77869.798633,42.493889,758.922689,1405.952143,-647.029454
201,2025-02-03,97681.101562,102514.171875,91242.890625,101405.421875,115400897748,102611.396484,99134.109375,78056.955410,48.798745,677.332065,1260.228128,-582.896063
202,2025-02-04,101398.718750,101745.617188,96208.109375,97871.820312,73002130211,102479.762891,98970.951406,78212.763730,33.348416,323.806156,1072.943733,-749.137577
203,2025-02-05,97878.007812,99113.210938,96174.828125,96615.445312,49125911241,102322.689844,98780.448281,78360.022715,35.095461,-57.086577,846.937671,-904.024248
204,2025-02-06,96610.640625,99168.609375,95707.351562,96593.296875,45302471947,101929.252734,98711.483438,78502.216582,34.207554,-356.623077,606.225522,-962.848599
205,2025-02-07,96581.320312,100154.140625,95653.882812,96529.085938,55741290456,101535.303516,98692.246094,78646.935762,31.600562,-592.360831,366.508251,-958.869082
206,2025-02-08,96533.257812,96877.804688,95702.492188,96482.453125,22447526395,101304.945703,98666.776563,78799.709668,31.682413,-774.025098,138.401581,-912.426679
207,2025-02-09,96481.312500,97325.281250,94745.257812,96500.093750,27732901800,101029.117188,98652.283906,78955.349473,34.888699,-906.126623,-70.504060,-835.622564
208,2025-02-10,96499.460938,98333.218750,95320.843750,97437.554688,40078962391,100593.681641,98698.936250,79113.651113,38.821196,-924.515532,-241.306354,-683.209178
209,2025-02-11,97438.132812,98492.898438,94875.039062,95747.429688,37488783272,100198.399609,98720.160000,79252.827949,37.151240,-1063.211793,-405.687442,-657.524351


In [ ]:
features = ['Open', 'High', 'Low', 'Close', 'Volume', 'MA_20', 'MA_50', 'MA_200', 'RSI', 'MACD', 'Signal', 'Histogram']
df = df.tail(10)
df = df.set_index('Date')
x = df[features].values  # فقط مقادیر عددی ستون‌های موردنظر

scaler = MinMaxScaler(feature_range=(0, 1))
x = scaler.fit_transform(np.array(x))
X_input = x.reshape(1, 10, 12)
X_input.shape
model_path = "pred_BTC_ind.h5"

try:
    model = load_model(
        model_path,
        custom_objects={'mse': MeanSquaredError()}  # استفاده از کلاس MeanSquaredError
    )
    print("Model loaded successfully.")
except Exception as e:
    print(f"Error loading model: {e}")

target_columns = [1, 2, 3]

# پیش‌بینی مقادیر با داده تست
y_pred_scaled = model.predict(X_input)

# ساخت آرایه کامل با شکل (نمونه‌ها, تمام ویژگی‌ها) برای inverse transform
dummy_array = np.zeros((y_pred_scaled.shape[0], x.shape[1]))  # ایجاد آرایه با ابعاد مشابه داده‌های ورودی
dummy_array[:, target_columns] = y_pred_scaled  # جایگذاری پیش‌بینی‌های مدل

# تبدیل به مقیاس واقعی
y_pred_real = scaler.inverse_transform(dummy_array)[:, target_columns]

# نمایش نتایج معکوس مقیاس شده
predicted_prices = pd.DataFrame(y_pred_real, columns=['Predicted High', 'Predicted Low', 'Predicted Close'])
print(predicted_prices)

(10, 12)